# Solving a 2nd order ODE with a feed-forward neural network using Autograd

Ordinary differential equations (ODEs) are crucial for modeling physical, chemical, financial, etc phenomena. Here, a neural network is used to numerically calculate to approximate a solution 2nd order ODE. The finite-difference method, of course, is a more reliable method for calculating solutions; however, the use of neural networks could prove useful in certain situations. 

In this notebook, a multi-layer perceptron neural network is used. 

Consider the following 2nd order ODE:

\begin{align}
\frac{d^2y(t)}{dt^2} = -k^2 y(t) 
\end{align}

The analytical solution to this ODE is $y(t) = \sin(kx)$ where $$y(0) = 0$$ $$y'(0) = k$$


References:<br>
[1] J. Kitchin, example of solving 1D ODE with NN http://kitchingroup.cheme.cmu.edu/blog/2017/11/28/Solving-ODEs-with-a-neural-network-and-autograd/ <br>
[2] Sine activation functions https://openreview.net/pdf?id=Sks3zF9eg <br>
[3] Long short-term memory neural network (not implemented here) https://link.springer.com/chapter/10.1007/978-3-319-47054-2_10/fulltext.html <br>
[4] Solving DE using neural networks https://becominghuman.ai/neural-networks-for-solving-differential-equations-fa230ac5e04c


In [1]:
# Import necessary libararies / functions
!pip install autograd

from autograd import grad, elementwise_grad
from autograd.misc.optimizers import adam
import autograd.numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import math

# Possible activation functions
def swish(x):
    return x / (1.0 + np.exp(-x))

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))
  
def periodic(x):
    return np.sin(x)
  
def tanh(x):
    return np.tanh(x)

    100% |████████████████████████████████| 829kB 582kB/s 
  Running setup.py bdist_wheel for autograd ... - \ | done
  Stored in directory: /Users/brentdevetter/Library/Caches/pip/wheels/72/6f/c2/40f130cca2c91f31d354bf72de282922479c09ce0b7853c4c5
  Running setup.py bdist_wheel for future ... - \ | / - done
  Stored in directory: /Users/brentdevetter/Library/Caches/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built autograd future
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


As described in reference [4], the ODE is solved by minimizing an objective function.  <br>

\begin{align}
z_{eq} = \frac{d^2y(t)}{dt^2} + k^2 y(t) = 0 \\ 
ic = y(0) - 0 = 0 \\
ic_2  = \frac{dy(0)}{dt} - k = 0 \\
\end{align}

\begin{align}
\Psi = z_{eq}^2 + ic^2 + ic_2^2 \\ 
\end{align}

Where $\Psi$ is minimized. First we attempt calculating the solution to this ODE using a 3-layer neural network with 100 hidden nodes. 
    

In [6]:
# For simplicity, assume k is 1
k = 1
  
t = np.linspace(0, math.pi*3).reshape((-1, 1))

# ODE function using a swish activation function
def ode_func(params, inputs):
  for W, b in params:
    outputs = np.dot(inputs, W) + b          
    inputs = swish(outputs)                   
  return outputs

# ODE function using a periodic (sine) activation function 
def ode_func_periodic(params, inputs):
  for W, b in params:
    outputs = np.dot(inputs, W) + b          
    inputs = periodic(outputs)              
  return outputs


# Derivatives
deriv_ode_func = elementwise_grad(ode_func, 1)
deriv_2_ode_func = elementwise_grad(deriv_ode_func, 1)

deriv_ode_func_periodic = elementwise_grad(ode_func_periodic, 1)
deriv_2_ode_func_periodic = elementwise_grad(deriv_ode_func_periodic, 1)

def setup_init_nn(scale, layer_sizes):
  rs = npr.RandomState(0)
  
  return [ (rs.randn(insize, outsize) * scale, # weight matrix
            rs.randn(outsize) * scale)         # bias vector
           for insize, outsize in zip(layer_sizes[:-1], layer_sizes[1:])]

# This function is minimized
def objective(params, step):
  zeq = deriv_2_ode_func(params, t) + k*k * ode_func(params, t)
  ic = deriv_ode_func(params, 0.0) - k
  ic2 = deriv_2_ode_func(params, 0.0)
  return np.mean(zeq**2) + ic**2 + ic2**2

# This function is minimized (periodic)
def objective_periodic(params, step):
  zeq = deriv_2_ode_func_periodic(params, t) + k*k * ode_func_periodic(params, t)
  ic = deriv_ode_func_periodic(params, 0.0) - k
  ic2 = deriv_2_ode_func_periodic(params, 0.0)
  return np.mean(zeq**2) + ic**2 + ic2**2

def callback(params, step, g):
  if step % 1000 == 0:
    print("Iteration: {0:3d} Objective: {1}".format(step, objective(params, step)))

params_swish = setup_init_nn(0.1, layer_sizes = [1, 100, 1])  
params_swish = adam(grad(objective), params_swish, step_size = 0.001, num_iters = 4000, callback = callback)

params_periodic = setup_init_nn(0.1, layer_sizes = [1, 100, 1])  
params_periodic = adam(grad(objective_periodic), params_periodic, step_size = 0.001, num_iters = 4000, callback = callback)

Iteration:   0 Objective: 0.9326984741285588
Iteration: 1000 Objective: 0.04799555951089569
Iteration: 2000 Objective: 0.02916355931368044
Iteration: 3000 Objective: 0.02655550884375522
Iteration:   0 Objective: 0.9326984741285588
Iteration: 1000 Objective: 0.5572130997130831
Iteration: 2000 Objective: 0.8146232550700254
Iteration: 3000 Objective: 0.9092899701086645


In [8]:
tfit = np.linspace(0, math.pi*6).reshape(-1, 1)
import matplotlib.pyplot as plt
plt.plot(tfit, ode_func(params_swish, tfit), 'r--', label='NN Solution (swish)')
plt.plot(tfit, ode_func_periodic(params_periodic, tfit), label='NN Solution (sine)')
plt.plot(tfit, np.sin(k*tfit), 'k', label='Analytic Solution ')
plt.plot()
plt.legend()
plt.xlabel('t')
plt.ylabel('$y(t)$')   
plt.xlim([0, math.pi*6])
plt.ylim([-4, 4])

(-4, 4)

The above figure shows that using a sine activation function results in a significantly better fit to data beyond the interval $[0, 3\pi]$, which the neural network was trained. Of course, this is a tad artificial because the analytic solution is sinusoidal. Additional experimention is necessary to determine how well an semi-arbitrary periodic function behaves with respect to different activation functions. 